# Bipolar Junction Transistor (BJT)

## Current-Controlled Amplification

The BJT is a three-terminal device where a small base current controls a larger collector current. It remains important for analog circuits and high-frequency applications.

**Learning Objectives:**
- Understand BJT operation (NPN and PNP)
- Simulate common-emitter characteristics with PADRE
- Generate Gummel plots from simulation
- Extract current gain (beta) from simulated data

In [ ]:
# Setup: Load PADRE environment (required on nanoHUB)
# This cell loads the PADRE simulator into your environment.
# If running locally with PADRE already in your PATH, this will be skipped gracefully.

from nanohubpadre import use

# Load the PADRE simulator environment
%use padre-2.4E-r15

print("PADRE environment setup complete.")

---

## 1. BJT Physics

### 1.1 Structure (NPN)

```
   Emitter (N+)  |  Base (P)  |  Collector (N)
   ============  |  ========  |  =============
   High doping   |  Thin,     |  Low doping
                 |  moderate  |
                 |  doping    |
```

### 1.2 Key Parameters

- **Current gain** ($\beta$ or $h_{FE}$):
  $$\beta = \frac{I_C}{I_B}$$

- **Collector current**:
  $$I_C = I_S \exp\left(\frac{V_{BE}}{V_T}\right)$$

- **Base current**:
  $$I_B = \frac{I_C}{\beta}$$

- **Early voltage** ($V_A$): Models output resistance
  $$I_C = I_S \exp\left(\frac{V_{BE}}{V_T}\right)\left(1 + \frac{V_{CE}}{V_A}\right)$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nanohubpadre import create_bjt

# Physical constants
q = 1.6e-19
kT = 0.0259  # Thermal voltage at 300K

print("BJT Parameters:")
print("="*40)
print(f"Thermal voltage VT = {kT*1000:.1f} mV")
print(f"Typical beta (hFE) = 50-300")
print(f"Typical Early voltage VA = 50-200 V")

---

## 2. Creating and Running a BJT Simulation

Let's create an NPN transistor and run PADRE to examine its characteristics.

In [ ]:
# Create NPN transistor
sim_npn = create_bjt(
    # Geometry
    emitter_width=1.0,      # Emitter region width (um)
    base_width=0.3,         # Base region width (um) - thin!
    collector_width=2.0,    # Collector region width (um)
    device_depth=1.0,
    
    # Mesh
    nx=150,
    ny=30,
    
    # Doping
    emitter_doping=1e20,    # N+ emitter
    base_doping=1e17,       # P base
    collector_doping=1e16,  # N collector
    device_type='npn',
    
    # Models
    temperature=300,
    srh=True,
    auger=True,
    bgn=True,
    
    # Output
    log_bands_eq=True
)

print("NPN BJT Configuration:")
print("="*40)
print("Emitter (N+): 1e20 cm⁻³")
print("Base (P): 1e17 cm⁻³, 0.3 μm wide")
print("Collector (N): 1e16 cm⁻³")

In [ ]:
# View generated deck
print("PADRE Input Deck:")
print("="*60)
print(sim_npn.generate_deck())

In [ ]:
# Run equilibrium simulation
print("Running BJT equilibrium simulation...")
result_eq = sim_npn.run()

if result_eq.returncode == 0:
    print("Simulation completed successfully!")
    print(f"Output directory: {sim_npn.working_dir}")
else:
    print("Simulation failed!")
    print(result_eq.stderr)

In [ ]:
# Plot equilibrium band diagram
print("\nAvailable outputs:")
print(sim_npn.outputs.summary())

In [ ]:
# Plot band diagram
try:
    sim_npn.plot_band_diagram(title="NPN BJT Equilibrium Band Diagram")
except Exception as e:
    print(f"Could not plot band diagram: {e}")

---

## 3. Gummel Plot (Ic, Ib vs Vbe)

The Gummel plot shows collector and base currents vs base-emitter voltage on a semi-log scale. This is the primary characterization tool for BJTs.

In [ ]:
# Gummel plot simulation
sim_gummel = create_bjt(
    emitter_width=1.0,
    base_width=0.3,
    collector_width=2.0,
    device_depth=1.0,
    
    emitter_doping=1e20,
    base_doping=1e17,
    collector_doping=1e16,
    device_type='npn',
    
    temperature=300,
    srh=True,
    auger=True,
    bgn=True,
    
    # Enable I-V logging
    log_iv=True,
    iv_file="gummel",
    
    # Gummel plot: sweep Vbe at fixed Vce
    gummel_sweep=(0.3, 0.85, 0.02),  # Vbe: 0.3 to 0.85V
    gummel_vce=2.0                    # Fixed Vce = 2V
)

print("Gummel Plot Simulation Configuration:")
print("="*40)
print("Vbe sweep: 0.3V to 0.85V")
print("Vce = 2.0V (fixed)")
print(f"Number of bias points: {int((0.85-0.3)/0.02) + 1}")

In [ ]:
# Run the Gummel plot simulation
print("Running Gummel plot simulation...")
print("(This may take a few minutes)")
result_gummel = sim_gummel.run()

if result_gummel.returncode == 0:
    print("Simulation completed successfully!")
else:
    print("Simulation failed!")
    print(result_gummel.stderr)

In [ ]:
# Plot the Gummel plot from simulation
try:
    iv_data = sim_gummel.get_iv_data()
    
    # Get Vbe, Ic, and Ib
    # Electrode assignments: 1=Emitter, 2=Base, 3=Collector (typical)
    Vbe, Ic = iv_data.get_gummel_data(base_electrode=2, collector_electrode=3)
    _, Ib = iv_data.get_gummel_data(base_electrode=2, collector_electrode=2)  # Base current
    
    Vbe = np.array(Vbe)
    Ic = np.abs(np.array(Ic))
    Ib = np.abs(np.array(Ib))
    
    # Create Gummel plot
    plt.figure(figsize=(10, 6))
    
    plt.semilogy(Vbe, Ic, 'b-o', linewidth=2, markersize=4, label='Ic (Collector)')
    plt.semilogy(Vbe, Ib, 'r-s', linewidth=2, markersize=4, label='Ib (Base)')
    
    plt.xlabel('Base-Emitter Voltage Vbe (V)', fontsize=12)
    plt.ylabel('Current (A)', fontsize=12)
    plt.title('Gummel Plot (PADRE Simulation)', fontsize=14)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xlim(0.3, 0.9)
    
    # Add slope annotation
    plt.annotate('Slope = q/kT\n(60 mV/decade)', xy=(0.5, 1e-9), fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    print("\nGummel Plot Analysis:")
    print("="*40)
    print(f"Vbe range: {Vbe[0]:.2f}V to {Vbe[-1]:.2f}V")
    print(f"Ic range: {Ic.min():.3e} A to {Ic.max():.3e} A")
    print(f"Ib range: {Ib.min():.3e} A to {Ib.max():.3e} A")
    
except Exception as e:
    print(f"Could not plot Gummel data: {e}")
    print("Trying built-in method...")
    sim_gummel.plot_gummel()

In [ ]:
# Extract current gain (beta) from simulation
try:
    # Calculate beta at each bias point
    mask = (Ic > 1e-12) & (Ib > 1e-12)  # Valid data points
    beta = Ic[mask] / Ib[mask]
    Vbe_valid = Vbe[mask]
    Ic_valid = Ic[mask]
    
    # Plot beta vs Ic
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Beta vs Vbe
    ax1.plot(Vbe_valid, beta, 'g-o', linewidth=2, markersize=4)
    ax1.set_xlabel('Vbe (V)', fontsize=12)
    ax1.set_ylabel('Current Gain β', fontsize=12)
    ax1.set_title('β vs Vbe', fontsize=14)
    ax1.grid(True, alpha=0.3)
    
    # Beta vs Ic (more common representation)
    ax2.semilogx(Ic_valid, beta, 'g-o', linewidth=2, markersize=4)
    ax2.set_xlabel('Collector Current Ic (A)', fontsize=12)
    ax2.set_ylabel('Current Gain β', fontsize=12)
    ax2.set_title('β vs Ic', fontsize=14)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Report statistics
    print("\nCurrent Gain Analysis:")
    print("="*40)
    print(f"Peak β = {beta.max():.1f}")
    print(f"β at Ic=1mA ≈ {beta[np.argmin(np.abs(Ic_valid - 1e-3))]:.1f}" if any(Ic_valid > 1e-3) else "Ic < 1mA")
    print(f"Average β = {beta.mean():.1f}")
    
    # Extract ideality factor
    mask_ideal = (Ic > 1e-10) & (Ic < 1e-5)
    if np.sum(mask_ideal) > 2:
        coeffs = np.polyfit(Vbe[mask_ideal], np.log(Ic[mask_ideal]), 1)
        n_ideality = 1 / (kT * coeffs[0])
        print(f"Ideality factor n ≈ {n_ideality:.2f}")
        
except Exception as e:
    print(f"Could not extract beta: {e}")

---

## 4. Common-Emitter Output Characteristics (Ic vs Vce)

The output characteristic shows collector current vs collector-emitter voltage at different base currents or base voltages.

In [ ]:
# Common-emitter output characteristics
sim_ce = create_bjt(
    emitter_width=1.0,
    base_width=0.3,
    collector_width=2.0,
    device_depth=1.0,
    
    emitter_doping=1e20,
    base_doping=1e17,
    collector_doping=1e16,
    device_type='npn',
    
    temperature=300,
    srh=True,
    bgn=True,
    
    # Enable I-V logging
    log_iv=True,
    iv_file="ic_vce",
    
    # Output characteristic: sweep Vce at fixed Vbe
    vbe=0.7,                      # Base-emitter voltage
    vce_sweep=(0.1, 3.0, 0.1)     # Vce: 0.1 to 3V
)

print("Common-Emitter Output Characteristic Configuration:")
print("="*40)
print("Vbe = 0.7V (fixed)")
print("Vce sweep: 0.1V to 3.0V")

In [ ]:
# Run the output characteristic simulation
print("Running common-emitter output simulation...")
result_ce = sim_ce.run()

if result_ce.returncode == 0:
    print("Simulation completed successfully!")
else:
    print("Simulation failed!")

In [ ]:
# Plot output characteristic
try:
    iv_ce = sim_ce.get_iv_data()
    Vce, Ic_ce = iv_ce.get_output_characteristic(collector_electrode=3)
    
    Vce = np.array(Vce)
    Ic_ce = np.abs(np.array(Ic_ce))
    
    plt.figure(figsize=(10, 6))
    plt.plot(Vce, Ic_ce * 1e3, 'b-o', linewidth=2, markersize=4, label='Vbe = 0.7V')
    
    plt.xlabel('Collector-Emitter Voltage Vce (V)', fontsize=12)
    plt.ylabel('Collector Current Ic (mA)', fontsize=12)
    plt.title('BJT Common-Emitter Output Characteristic (PADRE)', fontsize=14)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xlim(0, 3)
    plt.ylim(0, None)
    
    plt.tight_layout()
    plt.show()
    
    # Extract Early voltage from slope
    if len(Vce) > 5:
        # Use linear region (Vce > 0.5V)
        mask = Vce > 0.5
        if np.sum(mask) > 2:
            coeffs = np.polyfit(Vce[mask], Ic_ce[mask], 1)
            slope = coeffs[0]
            Ic_0 = coeffs[1]
            if slope > 0:
                Va = -Ic_0 / slope  # Early voltage
                print(f"\nExtracted Early voltage: Va ≈ {abs(Va):.1f} V")
    
except Exception as e:
    print(f"Could not plot output characteristic: {e}")

In [ ]:
# Simulate output characteristics at multiple Vbe values
Vbe_values = [0.65, 0.70, 0.75, 0.80]
ce_results = {}

print("Simulating output characteristics at multiple Vbe:")
print("="*50)

for Vbe in Vbe_values:
    print(f"\nVbe = {Vbe}V...")
    
    sim = create_bjt(
        emitter_width=1.0,
        base_width=0.3,
        collector_width=2.0,
        emitter_doping=1e20,
        base_doping=1e17,
        collector_doping=1e16,
        device_type='npn',
        temperature=300,
        srh=True,
        log_iv=True,
        iv_file=f"ce_vbe{int(Vbe*100)}",
        vbe=Vbe,
        vce_sweep=(0.1, 3.0, 0.1)
    )
    
    result = sim.run()
    
    if result.returncode == 0:
        try:
            iv = sim.get_iv_data()
            Vce, Ic = iv.get_output_characteristic(collector_electrode=3)
            ce_results[Vbe] = (np.array(Vce), np.abs(np.array(Ic)))
            print(f"  Success")
        except:
            print(f"  Could not parse data")
    else:
        print(f"  Failed")

In [ ]:
# Plot family of output characteristics
if ce_results:
    plt.figure(figsize=(10, 6))
    
    colors = ['blue', 'green', 'red', 'purple']
    
    for i, (Vbe, (Vce, Ic)) in enumerate(ce_results.items()):
        color = colors[i % len(colors)]
        plt.plot(Vce, Ic * 1e3, color=color, linewidth=2, label=f'Vbe = {Vbe}V')
    
    plt.xlabel('Collector-Emitter Voltage Vce (V)', fontsize=12)
    plt.ylabel('Collector Current Ic (mA)', fontsize=12)
    plt.title('BJT Output Characteristics (PADRE Simulation)', fontsize=14)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xlim(0, 3)
    plt.ylim(0, None)
    
    # Add Early voltage annotation
    plt.annotate('Slopes indicate\nEarly voltage VA', xy=(2.5, 0.5), fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    print("\nOutput characteristic analysis:")
    print("- Active region: Ic relatively constant vs Vce")
    print("- Slope = Ic/VA (Early effect)")
    print("- Higher Vbe → Exponentially higher Ic")

---

## 5. Base Width Effect

The base width is critical for BJT performance. Let's explore its effect using PADRE simulations.

In [ ]:
# Compare different base widths
base_widths = [0.2, 0.3, 0.5]  # microns
width_results = {}

print("Base Width Comparison:")
print("="*50)

for Wb in base_widths:
    print(f"\nWb = {Wb} μm...")
    
    sim = create_bjt(
        emitter_width=1.0,
        base_width=Wb,
        collector_width=2.0,
        emitter_doping=1e20,
        base_doping=1e17,
        collector_doping=1e16,
        device_type='npn',
        temperature=300,
        srh=True,
        log_iv=True,
        iv_file=f"gummel_Wb{int(Wb*10)}",
        gummel_sweep=(0.4, 0.8, 0.02),
        gummel_vce=2.0
    )
    
    result = sim.run()
    
    if result.returncode == 0:
        try:
            iv = sim.get_iv_data()
            Vbe, Ic = iv.get_gummel_data(base_electrode=2, collector_electrode=3)
            _, Ib = iv.get_gummel_data(base_electrode=2, collector_electrode=2)
            
            Vbe = np.array(Vbe)
            Ic = np.abs(np.array(Ic))
            Ib = np.abs(np.array(Ib))
            
            # Calculate beta
            mask = (Ic > 1e-12) & (Ib > 1e-12)
            beta_avg = np.mean(Ic[mask] / Ib[mask]) if np.sum(mask) > 0 else 0
            
            width_results[Wb] = {
                'Vbe': Vbe, 'Ic': Ic, 'Ib': Ib, 'beta': beta_avg
            }
            print(f"  Success, β_avg ≈ {beta_avg:.1f}")
        except:
            print(f"  Could not parse data")
    else:
        print(f"  Failed")

In [ ]:
# Plot base width comparison
if width_results:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    colors = ['blue', 'green', 'red']
    
    for i, (Wb, data) in enumerate(width_results.items()):
        color = colors[i % len(colors)]
        label = f'Wb = {Wb} μm (β≈{data["beta"]:.0f})'
        ax1.semilogy(data['Vbe'], data['Ic'], color=color, linewidth=2, label=label)
    
    ax1.set_xlabel('Vbe (V)', fontsize=12)
    ax1.set_ylabel('Ic (A)', fontsize=12)
    ax1.set_title('Gummel Plot: Effect of Base Width', fontsize=14)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Beta vs base width
    Wb_list = list(width_results.keys())
    beta_list = [data['beta'] for data in width_results.values()]
    
    ax2.plot(Wb_list, beta_list, 'go-', linewidth=2, markersize=10)
    ax2.set_xlabel('Base Width (μm)', fontsize=12)
    ax2.set_ylabel('Current Gain β', fontsize=12)
    ax2.set_title('β vs Base Width', fontsize=14)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\nKey observation:")
    print("Thinner base → Higher β (less base recombination)")
    print("β ~ 1/Wb for narrow base transistors")

---

## 6. Temperature Effects

BJT characteristics are strongly temperature-dependent.

In [ ]:
# Compare different temperatures
temperatures = [300, 350, 400]  # Kelvin
temp_results = {}

print("Temperature Comparison:")
print("="*50)

for T in temperatures:
    print(f"\nT = {T} K...")
    
    sim = create_bjt(
        emitter_width=1.0,
        base_width=0.3,
        collector_width=2.0,
        emitter_doping=1e20,
        base_doping=1e17,
        collector_doping=1e16,
        device_type='npn',
        temperature=T,
        srh=True,
        log_iv=True,
        iv_file=f"gummel_T{T}",
        gummel_sweep=(0.3, 0.8, 0.02),
        gummel_vce=2.0
    )
    
    result = sim.run()
    
    if result.returncode == 0:
        try:
            iv = sim.get_iv_data()
            Vbe, Ic = iv.get_gummel_data(base_electrode=2, collector_electrode=3)
            temp_results[T] = (np.array(Vbe), np.abs(np.array(Ic)))
            print(f"  Success")
        except:
            print(f"  Could not parse data")
    else:
        print(f"  Failed")

In [ ]:
# Plot temperature comparison
if temp_results:
    plt.figure(figsize=(10, 6))
    
    colors = ['blue', 'green', 'red']
    
    for i, (T, (Vbe, Ic)) in enumerate(temp_results.items()):
        color = colors[i % len(colors)]
        kT_val = T * 8.617e-5  # kT in eV
        plt.semilogy(Vbe, Ic, color=color, linewidth=2, 
                     label=f'T = {T} K (kT = {kT_val*1000:.1f} mV)')
    
    plt.xlabel('Base-Emitter Voltage Vbe (V)', fontsize=12)
    plt.ylabel('Collector Current Ic (A)', fontsize=12)
    plt.title('Temperature Effect on BJT (PADRE Simulation)', fontsize=14)
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\nTemperature effects:")
    print("- Higher T → Higher Is (saturation current)")
    print("- Slope changes: q/kT decreases with T")
    print("- Vbe for same Ic decreases ~2mV/°C")

---

## 7. Exercises

### Exercise 1: PNP Transistor
Create a PNP transistor and compare its characteristics to the NPN.

In [ ]:
# Exercise 1: PNP transistor
sim_pnp = create_bjt(
    emitter_width=1.0,
    base_width=0.3,
    collector_width=2.0,
    emitter_doping=1e20,
    base_doping=1e17,
    collector_doping=1e16,
    device_type='pnp',  # PNP transistor
    temperature=300,
    srh=True,
    log_iv=True,
    iv_file="pnp_gummel",
    gummel_sweep=(-0.3, -0.8, -0.02),  # Negative Vbe for PNP
    gummel_vce=-2.0                     # Negative Vce for PNP
)

print("Running PNP simulation...")
result_pnp = sim_pnp.run()

if result_pnp.returncode == 0:
    print("PNP simulation completed!")
    
    try:
        iv_pnp = sim_pnp.get_iv_data()
        Vbe_p, Ic_p = iv_pnp.get_gummel_data(base_electrode=2, collector_electrode=3)
        
        plt.figure(figsize=(10, 6))
        plt.semilogy(np.abs(Vbe_p), np.abs(Ic_p), 'r-o', linewidth=2, label='PNP Ic')
        
        plt.xlabel('|Vbe| (V)', fontsize=12)
        plt.ylabel('|Ic| (A)', fontsize=12)
        plt.title('PNP BJT Gummel Plot', fontsize=14)
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        print("\nPNP vs NPN:")
        print("- PNP: P-type emitter and collector, N-type base")
        print("- Holes are majority carriers")
        print("- Generally lower β (hole mobility < electron mobility)")
        print("- Requires negative Vbe and Vce")
        
    except Exception as e:
        print(f"Could not plot: {e}")
else:
    print("Simulation failed")

---

## Summary

In this notebook, you learned:

1. **BJT Structure**: Emitter, Base, Collector regions
2. **Running PADRE Simulations**: Using `create_bjt()` and `sim.run()`
3. **Gummel Plot**: Ic and Ib vs Vbe from PADRE simulation
4. **Output Characteristics**: Ic vs Vce from simulation
5. **Current Gain**: Extracting β from simulated data
6. **Design Parameters**: Base width effect on β
7. **Temperature Effects**: Impact on BJT characteristics

**Key Equations:**
- $I_C = I_S e^{V_{BE}/V_T}$
- $\beta = I_C / I_B$
- Early effect: $I_C = I_S e^{V_{BE}/V_T}(1 + V_{CE}/V_A)$

**Next**: [06 - Solar Cell](06_Solar_Cell.ipynb) - Photovoltaic devices